In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
from model import forecast 

c:\Users\Lev\miniconda3\envs\ds\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dates_to_test = pd.date_range(start='2023-07-19', end='2023-08-01').to_list()
store_to_test_mod1 = 'f7e6c85504ce6e82442c770f7c8606f0'
item_to_test_mod1 = {"sku": "4b083e10f6cd2f4360c5425d05be6386",
                     "group": "6512bd43d9caa6e02c990b0a82652dca",
                     "category": "e58cc5ca94270acaceed13bc82dfedf7",
                     "subcategory": "120e4b609af49ff7b17c3b90e7a28435"}

In [4]:
#test model2
store_to_test_mod2 = '16a5cdae362b8d27a1d8f8c7b78b4330'
item_to_test_mod2 = {"sku": "0045ebdb1069ff4b3dd3efe628c39cd3",
                     "group": "c74d97b01eae257e44aa9d5bade97baf",
                     "category": "c559da2ba967eb820766939a658022c8",
                     "subcategory": "130d817d8b3f616e97c555ec57946aec"}
# ''
# forecast(item_to_test_mod2, store_to_test_mod2, dates_to_test)

# #[2, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
# #[3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3]

In [5]:
sales = pd.read_csv('sales_db.csv', parse_dates=['date'])
sales.head()

,date,pr_sales_type_id,pr_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_units,pr_promo_sales_in_rub,pr_sku_id,st_id
0,2023-04-07,1,1.0,81.0,1.0,81.0,0376a60d9a7ce7965beddc4815588697,084a8a9aa8cced9175bd07bc44998e75
1,2023-04-11,1,1.0,81.0,1.0,81.0,0376a60d9a7ce7965beddc4815588697,084a8a9aa8cced9175bd07bc44998e75
2,2023-04-12,1,1.0,81.0,1.0,81.0,0376a60d9a7ce7965beddc4815588697,084a8a9aa8cced9175bd07bc44998e75
3,2023-04-29,0,1.0,146.0,0.0,0.0,0376a60d9a7ce7965beddc4815588697,084a8a9aa8cced9175bd07bc44998e75
4,2023-05-24,0,1.0,146.0,0.0,0.0,0376a60d9a7ce7965beddc4815588697,084a8a9aa8cced9175bd07bc44998e75


In [6]:
sales.shape

(661192, 8)

In [7]:
def get_history_by_store_item(sku, store, last_date):
    result = pd.DataFrame({'date': pd.date_range(start=(pd.to_datetime(last_date)-pd.Timedelta(days=20)), end=last_date)})
    from_bd = sales[(sales['pr_sku_id'] == sku) & (sales['st_id'] == store)][['date', 'pr_sales_in_units']]
    result = result.merge(from_bd, how='left', on='date')
    result = result.fillna(0)
    dates_list = result['date'].dt.strftime('%Y-%m-%d').to_list()
    sales_list = result['pr_sales_in_units'].to_list()
    dict_result = {dates_list[i]: sales_list[i] for i in range(len(dates_list))}
    return dict_result

In [8]:
get_history_by_store_item('4b083e10f6cd2f4360c5425d05be6386', 'f7e6c85504ce6e82442c770f7c8606f0', '2023-07-18')

{'2023-06-28': 79.0,
 '2023-06-29': 71.0,
 '2023-06-30': 101.0,
 '2023-07-01': 97.0,
 '2023-07-02': 65.0,
 '2023-07-03': 26.0,
 '2023-07-04': 36.0,
 '2023-07-05': 55.0,
 '2023-07-06': 80.0,
 '2023-07-07': 34.0,
 '2023-07-08': 41.0,
 '2023-07-09': 40.0,
 '2023-07-10': 18.0,
 '2023-07-11': 86.0,
 '2023-07-12': 86.0,
 '2023-07-13': 13.0,
 '2023-07-14': 0.0,
 '2023-07-15': 28.0,
 '2023-07-16': 57.0,
 '2023-07-17': 59.0,
 '2023-07-18': 37.0}

In [9]:
def get_history_by_store(store, last_date):
    result = pd.DataFrame({'date': pd.date_range(start=(pd.to_datetime(last_date)-pd.Timedelta(days=20)), end=last_date)})
    from_bd = sales.groupby(['st_id', 'date'])['pr_sales_in_units'].sum().reset_index()
    from_bd = from_bd[from_bd['st_id'] == store]
    result = result.merge(from_bd, how='left', on='date')
    result = result.fillna(0)
    dates_list = result['date'].dt.strftime('%Y-%m-%d').to_list()
    sales_list = result['pr_sales_in_units'].to_list()
    dict_result = {dates_list[i]: sales_list[i] for i in range(len(dates_list))}
    return dict_result

In [10]:
get_history_by_store('f7e6c85504ce6e82442c770f7c8606f0', '2023-07-18')

{'2023-06-28': 1795.0,
 '2023-06-29': 2004.0,
 '2023-06-30': 4042.0,
 '2023-07-01': 4355.0,
 '2023-07-02': 2907.0,
 '2023-07-03': 2233.0,
 '2023-07-04': 2207.0,
 '2023-07-05': 2426.0,
 '2023-07-06': 3099.0,
 '2023-07-07': 4462.0,
 '2023-07-08': 4129.0,
 '2023-07-09': 2659.0,
 '2023-07-10': 2190.0,
 '2023-07-11': 2761.0,
 '2023-07-12': 2756.0,
 '2023-07-13': 3089.0,
 '2023-07-14': 4580.0,
 '2023-07-15': 4760.0,
 '2023-07-16': 2805.0,
 '2023-07-17': 2192.0,
 '2023-07-18': 2304.0}

In [11]:
def get_history_by_item(sku, last_date):
    min_date = sales['date'].min()
    max_date = last_date
    result = pd.DataFrame({'date': pd.date_range(start=min_date, end=max_date)})
    from_bd = sales.groupby(['pr_sku_id', 'date'])[['pr_sales_in_units', 'pr_sales_in_rub']].sum().reset_index()
    from_bd = from_bd[from_bd['pr_sku_id'] == sku]
    result = result.merge(from_bd, how='left', on='date')
    result = result.fillna(0)
    dates_list = result['date'].dt.strftime('%Y-%m-%d').to_list()
    sales_units_list = result['pr_sales_in_units'].to_list()
    sales_rub_list = result['pr_sales_in_rub'].to_list()
    dict_result = {dates_list[i]: (sales_units_list[i], sales_rub_list[i]) for i in range(len(dates_list))}
    return dict_result

In [12]:
item_sales_history = get_history_by_item('4b083e10f6cd2f4360c5425d05be6386', '2023-07-18')
item_sales_history

{'2022-08-01': (146.0, 13717.0),
 '2022-08-02': (166.0, 15650.0),
 '2022-08-03': (204.0, 18569.0),
 '2022-08-04': (130.0, 13667.0),
 '2022-08-05': (179.0, 18039.0),
 '2022-08-06': (67.0, 6584.0),
 '2022-08-07': (41.0, 3755.0),
 '2022-08-08': (13.0, 1691.0),
 '2022-08-09': (86.0, 8445.0),
 '2022-08-10': (97.0, 9938.0),
 '2022-08-11': (280.0, 22739.0),
 '2022-08-12': (108.0, 10523.0),
 '2022-08-13': (129.0, 13565.0),
 '2022-08-14': (117.0, 12555.0),
 '2022-08-15': (137.0, 14333.0),
 '2022-08-16': (81.0, 7913.0),
 '2022-08-17': (99.0, 9368.0),
 '2022-08-18': (99.0, 10103.0),
 '2022-08-19': (251.0, 24890.0),
 '2022-08-20': (205.0, 20191.0),
 '2022-08-21': (206.0, 20971.0),
 '2022-08-22': (123.0, 12307.0),
 '2022-08-23': (112.0, 10610.0),
 '2022-08-24': (74.0, 7363.0),
 '2022-08-25': (201.0, 17127.0),
 '2022-08-26': (235.0, 22568.0),
 '2022-08-27': (110.0, 11405.0),
 '2022-08-28': (46.0, 4577.0),
 '2022-08-29': (50.0, 4965.0),
 '2022-08-30': (136.0, 13555.0),
 '2022-08-31': (143.0, 14369.0)

In [13]:
len(item_sales_history)

352

In [14]:
stats_by_store_item1 = get_history_by_store_item('4b083e10f6cd2f4360c5425d05be6386', 'f7e6c85504ce6e82442c770f7c8606f0', '2023-07-18')
stats_by_store1 = get_history_by_store('f7e6c85504ce6e82442c770f7c8606f0', '2023-07-18')
stats_by_item1 = get_history_by_item('4b083e10f6cd2f4360c5425d05be6386', '2023-07-18')

In [15]:
forecast(item_to_test_mod1, store_to_test_mod1, stats_by_store_item1, stats_by_store1, stats_by_item1, dates_to_test)

#[31, 30, 36, 37, 30, 23, 22, 22, 21, 27, 29, 22, 18, 18]
#[35, 32, 38, 37, 29, 25, 29, 27, 28, 34, 33, 27, 23, 26]
#[36, 28, 27, 28, 30, 30, 29, 26, 23, 22, 25, 25, 25, 25]

15:21:59 - cmdstanpy - INFO - Chain [1] start processing
15:21:59 - cmdstanpy - INFO - Chain [1] done processing


[36, 28, 27, 28, 30, 30, 29, 26, 23, 22, 25, 25, 25, 25]

In [18]:
stats_by_store_item2 = get_history_by_store_item('02713435f3587e2c81d8f6a9016763ea', '16a5cdae362b8d27a1d8f8c7b78b4330', '2023-07-18')
stats_by_store2 = get_history_by_store('16a5cdae362b8d27a1d8f8c7b78b4330', '2023-07-18')
stats_by_item2 = get_history_by_item('02713435f3587e2c81d8f6a9016763ea', '2023-07-18')

In [19]:
forecast(item_to_test_mod2, store_to_test_mod2, stats_by_store_item2, stats_by_store2, stats_by_item2, dates_to_test)

15:22:37 - cmdstanpy - INFO - Chain [1] start processing
15:22:37 - cmdstanpy - INFO - Chain [1] done processing


[3, 3, 3, 3, 3, 2, 2, 3, 4, 4, 4, 3, 3, 3]